# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra
using ASEconvert

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]
    unit_cell = periodic_system(lattice, atoms, positions)

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE, make the supercell
    # and then convert back ...
    supercell_ase = convert_ase(unit_cell) * pytuple((repeat, 1, 1))
    supercell     = pyconvert(AbstractSystem, supercell_ase)

    # Unfortunately right now the conversion to ASE drops the pseudopotential information,
    # so we need to reattach it:
    supercell = attach_psp(supercell, Al="hgh/lda/al-q3")

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(supercell; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
convert_ase(periodic_system(setup.model)).write("al_supercell.png")

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123


Python None

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298302296744                   -0.85    5.1         
  2   -8.300179573386       -2.73       -1.26    1.8    126ms
  3   -8.300432574829       -3.60       -1.87    2.0    134ms
  4   -8.300461854759       -4.53       -2.73    1.9    141ms
  5   -8.300464198747       -5.63       -3.12    2.9    218ms
  6   -8.300464404248       -6.69       -3.25    8.9    222ms
  7   -8.300464520903       -6.93       -3.40    4.8    186ms
  8   -8.300464577418       -7.25       -3.52    2.1    144ms
  9   -8.300464629201       -7.29       -3.75    1.8    149ms
 10   -8.300464631797       -8.59       -3.81    3.5    164ms
 11   -8.300464642305       -7.98       -4.10    2.9    229ms


In [4]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -16.64191841085                   -0.70    6.1         
  2   -16.67868613484       -1.43       -1.14    1.4    307ms
  3   -16.67921253201       -3.28       -1.88    3.0    342ms
  4   -16.67927804297       -4.18       -2.74    2.6    377ms
  5   -16.67928596478       -5.10       -3.10    4.0    434ms
  6   -16.67928620546       -6.62       -3.53    3.8    364ms
  7   -16.67928621922       -7.86       -4.02    1.9    331ms


In [5]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32568453642                   -0.56    7.2         
  2   -33.33272193156       -2.15       -1.00    1.0    1.06s
  3   -33.33411886598       -2.85       -1.74    4.0    1.31s
  4   -33.33428767995       -3.77       -2.65    2.2    1.14s
  5   -33.33583269463       -2.81       -2.41    5.9    1.59s
  6   -33.33694123308       -2.96       -2.50    8.1    1.50s
  7   -33.33694168967       -6.34       -2.50    2.4    1.09s
  8   -33.33674002126   +   -3.70       -2.22    3.5    1.42s
  9   -33.33666375823   +   -4.12       -2.15    1.0    1.01s
 10   -33.33657380608   +   -4.05       -2.09    1.0    1.04s
 11   -33.33650989798   +   -4.19       -2.05    1.0    1.01s
 12   -33.33645254479   +   -4.24      

When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298566877135                   -0.85    5.6         
  2   -8.300289247039       -2.76       -1.58    1.5    123ms
  3   -8.300442425844       -3.81       -2.71    2.1    112ms
  4   -8.300443809680       -5.86       -2.59    7.1    234ms
  5   -8.300464497815       -4.68       -3.39    1.0    106ms
  6   -8.300464616288       -6.93       -3.99    2.6    140ms
  7   -8.300464642518       -7.58       -4.45    3.1    175ms


In [7]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32693724054                   -0.56    7.2         
  2   -33.31322636538   +   -1.86       -1.27    1.5    947ms
  3   -12.61841543113   +    1.32       -0.41    6.5    2.00s
  4   -33.30765661738        1.32       -1.51    5.4    1.76s
  5   -33.29203004699   +   -1.81       -1.46    2.9    1.36s
  6   -32.88999047098   +   -0.40       -1.24    3.2    1.29s
  7   -33.31448589286       -0.37       -1.87    3.0    1.24s
  8   -33.33613110533       -1.66       -2.48    2.2    1.05s
  9   -33.33635879463       -3.64       -2.42    3.2    1.51s
 10   -33.33664539812       -3.54       -2.76    2.2    1.19s
 11   -33.33688629595       -3.62       -3.10    2.1    1.05s
 12   -33.33691594145       -4.53      

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.